In [66]:
import random


class Grammar:
    def __init__(self):
      #define the non-terminal symbols
        self.VN = {'S', 'A', 'B'}
      #define the terminal symbols
        self.VT = {'a', 'b', 'c', 'd'}
      #define the productions
        self.P = {
            'S': ['bS', 'dA'],
            'A': ['aA', 'dB', 'b'],
            'B': ['cB', 'a']
        }
        #define the start symbol
        self.start = 'S'

    #method to generate n number of strings according to given the rules
    def generate_strings(self, n=5):
        strings = []
        for i in range(n):
            string = ''
            stack = [self.start]
            while stack:
                symbol = stack.pop()
                if symbol in self.VT:
                    string += symbol
                elif symbol in self.VN:
                    productions = self.P[symbol]
                    production = random.choice(productions)
                    for s in reversed(production):
                        stack.append(s)
            strings.append(string)
        return strings

# Chomsky classification to identify the type of the grammar
    def chomsky_classifier(self):
        type1 = True
        type2 = True
        type3_right = True
        type3_left = True

        for rule in self.P.keys():
            if len(self.P) != 1:
                type2 = False
                break

            if all(len(p) < 1 for p in self.P):
                type1 = False
                break
                
        for i in self.VN:
            for production in self.P[i]:
                if (len(self.P) == 2 and self.P[1] not in self.VN and
                    self.P[0] not in self.VT) or len(self.P) > 2:
                    type3_right = False
                    break

                if (len(self.P) == 1 and self.P[0] not in self.VT) or (len(self.P) == 2 and self.P[0] not in self.VN) or len(self.P) > 2:
                    type3_left = False
                    break

                if any(len(lst) > 2 for lst in self.P) or any(len(lst) == 0 for lst in self.P):
                    type2 = False
                    break

                if len(self.P) == 0:
                    type1 = False
                    break

        if type3_right:
            return 'Type 3: Right linear regular grammar'
        elif type3_left:
            return 'Type 3: Left linear regular grammar'
        elif type2:
            return 'Type 2: Context-free grammar'
        elif type1:
            return 'Type 1: context-sensitive grammar'
        else:
            return 'Type 0: unrestrictive grammar'


grammar = Grammar()
identify_grammar_type = grammar.chomsky_classifier()
print(identify_grammar_type)

Type 3: Left linear regular grammar


In [ ]:
class FiniteAutomaton:
    def __init__(self):
        self.states = set()
        self.alphabet = set()
        self.transitions = {}
        self.start_state = None
        self.final_states = set()

def grammar_to_finite_automaton(grammar):
    fa = FiniteAutomaton()
    fa.states.add('q0')
    fa.start_state = 'q0'
    for symbol in grammar.VT:
        fa.alphabet.add(symbol)
    for variable in grammar.VN:
        for production in grammar.P[variable]:
            source_state = 'q' + str(hash(production))
            fa.states.add(source_state)
            if variable == grammar.start:
                fa.transitions[(fa.start_state, '')] = source_state
            for i in range(len(production)):
                if production[i] in grammar.VT:
                    fa.transitions[(source_state, production[i])] = 'q' + str(hash(production[:i] + '.' + production[i+1:]))
            if production[-1] in grammar.VN:
                fa.final_states.add(source_state)
    return fa

def accepts(fa, input_string):
    current_state = fa.start_state
    for symbol in input_string:
        if (current_state, symbol) in fa.transitions:
            current_state = fa.transitions[(current_state, symbol)]
        else:
            return False
    return current_state in fa.final_states

fa = FiniteAutomaton()
grammar_to_finite_automaton(g)
string = g.generate_strings()
for x in range(5):
  print(accepts(fa, string[x]))

False
False
False
False
False


In [81]:
class Conversion:
    def __init__(self, Q, sigma, delta, q0, F):
        self.Q = Q
        self.sigma = sigma
        self.delta = delta
        self.q0 = q0
        self.F = F

    def is_deterministic(self):
        if not self.Q or not self.sigma:
            return False

        for i in self.Q:
            for j in self.sigma:
                if (i, j) not in self.delta or self.delta[(i, j)] not in self.Q:
                    return False

        if self.q0 not in self.Q:
            return False

        if not set(self.F).issubset(set(self.Q)):
            return False
        return True

    def convert_to_dfa(self):
        if self.is_deterministic():
            return self

        dfa_states = set()
        dfa_accept_states = set()
        dfa_transitions = dict()
        state_queue = [frozenset([self.q0])]
        while state_queue:
            current_states = state_queue.pop(0)
            dfa_states.add(current_states)
            if any(state in self.F for state in current_states):
                dfa_accept_states.add(current_states)
            for symbol in self.sigma:
                next_states = set()
                for state in current_states:
                    next_states |= set(self.delta.get((state, symbol), set()))
                if next_states:
                    next_states = frozenset(next_states)
                    dfa_transitions[(current_states, symbol)] = next_states
                    if next_states not in dfa_states:
                        state_queue.append(next_states)

        dfa = Conversion(dfa_states, self.sigma, dfa_transitions, self.q0, dfa_accept_states)
        return dfa

    def convert_to_grammar(self):
        productions = dict()
        for state in self.Q:
            for symbol in self.sigma:
                next_states = self.delta.get((state, symbol), set())
                for next_state in next_states:
                    if next_state in self.F:
                        if state not in productions:
                            productions[state] = set()
                        productions[state].add(symbol)
                    else:
                        if next_state not in productions:
                            productions[next_state] = set()
                        productions[next_state].add(symbol + ' ' + state)
        start_symbol = self.q0
        if start_symbol in productions:
            productions['S'] = productions[start_symbol]
            del productions[start_symbol]
            for state in self.Q:
                for symbol in self.sigma:
                    next_states = self.delta.get((state, symbol), set())
                    for next_state in next_states:
                        if state in productions and symbol + state in productions[state]:
                            if next_state not in productions:
                                productions[next_state] = set()
                            productions[next_state].add(symbol + ' S')
        else:
            start_symbol = 'S'
            productions[start_symbol] = set()
            for accept_state in self.F:
                productions[start_symbol].add('eps' + accept_state)

        return start_symbol, productions

Q = {'q0', 'q1', 'q2', 'q3'}
sigma = {'a', 'b'}
delta =  {
        ('q0', 'a'): {'q0', 'q1'},
        ('q1', 'a'): {'q2'},
        ('q1', 'b'): {'q1'},
        ('q2', 'a'): {'q3'},
        ('q3', 'a'): {'q1'}
        }
q0 = 'q2'
F = {'q3'}
convesion = Conversion(Q, sigma, delta, q0, F)
print("Is this Automaton deterministic? ", convesion.is_deterministic())
dfa = convesion.convert_to_dfa()
print("States: ", dfa.Q)
print("Transition function: ", dfa.delta)
print("Initial state: ", dfa.q0)
print("Final states: ", dfa.F)

grammar = convesion.convert_to_grammar()
print("Regular grammar productions: ", grammar)

Is this Automaton deterministic?  False
States:  {frozenset({'q2'}), frozenset({'q1'}), frozenset({'q3'})}
Transition function:  {(frozenset({'q2'}), 'a'): frozenset({'q3'}), (frozenset({'q3'}), 'a'): frozenset({'q1'}), (frozenset({'q1'}), 'a'): frozenset({'q2'}), (frozenset({'q1'}), 'b'): frozenset({'q1'})}
Initial state:  q2
Final states:  {frozenset({'q3'})}
Regular grammar productions:  ('q2', {'q0': {'a q0'}, 'q1': {'a q0', 'b q1', 'a q3'}, 'S': {'a', 'a q1'}})
